In [1]:
using CairoMakie;
using GeophysicalFlows;

include("analysis/load_file.jl")
t, x, k = load_packet_analysis_file("data/2layer_test.jld2");

[ Info: FourierFlows will use 8 threads


In [2]:
using JLD2
file = jldopen("data/2layer_test.jld2");

nlayers = 2;
dev = CPU();
nx = file["grid/nx"];
Lx = file["grid/Lx"];
f₀ = file["params/f₀"];
g = file["params/g"];
H = file["params/H"];
ρ = [ρ for ρ in file["params/ρ"]];
prob = MultiLayerQG.Problem(nlayers, dev; nx, Lx, f₀, g, H, ρ, aliased_fraction=1/3)
grid = prob.grid;
vars = prob.vars;
params = prob.params;

In [88]:
Nsnapshots = length(keys(file["snapshots/t"]));
frames = keys(file["packets/t"])[2:end];
background_frames = keys(file["snapshots/t"]);
background_frame_index = 1;

q = Observable(Array(vars.q[:, :, 1]))
x = [];
y = [];
fig = Figure(resolution=(600, 600))
ax = Axis(fig[1, 1]; title = "q")
heatmap!(ax, grid.x, grid.y, q; colormap = :balance)
scatter!(ax, x, y);
record(fig, "top_layer_pv.mp4", frames; framerate = 18) do t
    MultiLayerQG.updatevars!(prob.vars, prob.params, prob.grid, file["snapshots/sol/$t"]) 
    q[] = vars.q[:,:,1]
end

"time_animation.mp4"

In [3]:
file

JLDFile C:\Users\ndefi\Documents\NYU\Research\JuliaRaytracing\data\2layer_test.jld2 (read-only)
 ├─📂 eqn
 │  ├─🔢 L
 │  ├─🔢 dims
 │  └─🔢 T
 ├─📂 clock
 │  └─🔢 dt
 ├─📂 grid
 │  ├─🔢 nx
 │  └─ ⋯ (5 more entries)
 └─ ⋯ (3 more entries)

In [4]:
keys(file)

6-element Vector{String}:
 "eqn"
 "clock"
 "grid"
 "params"
 "packets"
 "snapshots"

In [15]:
length(keys(file["snapshots/sol"]))

201

In [11]:
keys(file["snapshots"])

3-element Vector{String}:
 "t"
 "sol"
 "E"

In [ ]:
1024*